In [300]:
import pandas as pd
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import datetime
devengine = create_engine("sqlite:///C:/Git/tennis_atp/database/bets_sqllite.db")
time_now = datetime.datetime.now()
time_now_formatted = time_now.strftime("%Y-%m-%d")

In [301]:
pd.read_sql_query(
    f"Select DISTINCT * From results_hard",con=devengine,)

,Winner,Winner_Odds,Fav_Odds,Dog_Odds,Fav,Elo_Fav,Fav_Record,Fav_Games,Dog,Dog_Record,Dog_Games,fav_percent,dog_percent,Sex
0,Joris De Loore,1.27,1.27,3.48,Joris De Loore,Joris De Loore,90%,10,Denis Yevseyev,36%,11,0.900000,0.363636,Mens
1,Maxime Cressy,2.43,1.56,2.43,Emil Ruusuvuori,Emil Ruusuvuori,49%,47,Maxime Cressy,47%,32,0.489362,0.468750,Mens
2,Cem Ilkel,1.63,1.63,2.2,Cem Ilkel,Cem Ilkel,50%,20,Laurent Lokoli,70%,10,0.500000,0.700000,Mens
3,Ricardas Berankis,1.48,1.48,2.53,Ricardas Berankis,Ricardas Berankis,64%,11,Altug Celikbilek,53%,17,0.636364,0.529412,Mens
4,Arthur Fils,3.74,1.28,3.74,Roberto Bautista Agut,Roberto Bautista Agut,80%,10,Arthur Fils,45%,11,0.800000,0.454545,Mens
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6230,Zizou Bergs,1.79,1.79,1.96,Zizou Bergs,Ruben Bemelmans,20%,10,Ruben Bemelmans,50%,10,0.200000,0.500000,Mens
6231,Alexandre Muller,1.51,1.51,2.48,Alexandre Muller,Alexandre Muller,62%,13,Cem Ilkel,45%,11,0.615385,0.454545,Mens
6232,Borna Gojo,1.48,1.48,2.55,Borna Gojo,Borna Gojo,50%,14,Kimmer Coppejans,36%,11,0.500000,0.363636,Mens
6233,Enzo Couacaud,2.08,1.71,2.08,Jiri Vesely,Enzo Couacaud,70%,10,Enzo Couacaud,55%,11,0.700000,0.545455,Mens


In [302]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,fav_percent,dog_percent,Fav_Games,Dog_Games,Fav_Odds,Fav,sex,Dog_Odds From results_hard \
    where sex like 'Mens' --and Date > '2023-01-01'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Dog_Odds']=data['Dog_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)
#print(len(data[(data['Winner']==data['Fav'])&(data['Fav_Odds']>1.9)])/len(data[data['Fav_Odds']>1.9]))
#data[data['Fav_Odds']>1.8].to_csv('test.csv')
#data=data[data['Fav_Odds']>1.19]
#data=data[data['fav_percent']>0.3]
#data=data[data['dog_percent']>0.1]

In [303]:
mensserving=pd.read_csv('mensserving.csv')
combine = pd.merge(data, mensserving, how="left", left_on="Fav", right_on="Name")
combine2 = pd.merge(combine, mensserving, how="left", left_on="Dog", right_on="Name")
combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )
mens = combine2[
    [   "Winner","Dog","Winner_Odds","fav_percent","dog_percent","Fav_Games","Dog_Games","Fav_Odds",
        "Fav",
        # "Player_1_Odds",
        "Fav_Serve%",
        "Dog_Return%",
       
        # "Player_2_Odds",
        "Dog_Serve%",
        "Fav_Return%","Dog_Odds"
    ]]

In [304]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        k=x+0.1
        l=y-0.1
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['dog_percent']<k)&(mens['fav_percent']<y)&(mens['fav_percent']>l)]
        mensfiltertxt=f"print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>{x.round(1)})&(final_hard['dog_percent']<{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})])"
        
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:

            print(mensfiltertxt)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        k=x+0.1
        l=y+0.1
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['dog_percent']>k)&(mens['fav_percent']<y)&(mens['fav_percent']>l)]
        mensfiltertxt=f"print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']<{x.round(1)})&(final_hard['dog_percent']>{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})])"

        
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            
            print(mensfiltertxt)
            #print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            #print(len(mensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
        k=x+0.1
        l=y+0.1

        mensfilter=mens[(mens['dog_percent']>x)&(mens['dog_percent']<k)&(mens['fav_percent']>y)&(mens['fav_percent']<l)]
        mensfiltertxt=f"print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>{x.round(1)})&(final_hard['dog_percent']<{k.round(1)})&(final_hard['fav_percent']>{y.round(1)})&(final_hard['fav_percent']<{l.round(1)})])"

        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            print(mensfiltertxt)
            #print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            #print(len(mensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['fav_percent']>y)]
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))    

><
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>0.0)&(final_hard['dog_percent']<0.1)&(final_hard['fav_percent']<0.6)&(final_hard['fav_percent']>0.5)])
0.7142857142857143
7
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>0.4)&(final_hard['dog_percent']<0.5)&(final_hard['fav_percent']<0.2)&(final_hard['fav_percent']>0.1)])
1.0
2
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>0.5)&(final_hard['dog_percent']<0.6)&(final_hard['fav_percent']<0.3)&(final_hard['fav_percent']>0.2)])
0.6923076923076923
13
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>0.5)&(final_hard['dog_percent']<0.6)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)])
0.5263157894736842
19
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='Mens')&(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<

In [305]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['fav_percent']<y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:

            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['fav_percent']<y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']>x)&(mens['fav_percent']>y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        mensfilter=mens[(mens['dog_percent']<x)&(mens['fav_percent']>y)&(mens['Fav_Odds']>1.19)&(mens['Fav_Odds']<1.3)]
        if len(mensfilter)>10 and len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter) > 0.9:
            print(x,y)
            print(len(mensfilter[mensfilter['Winner']!=mensfilter['Dog']])/len(mensfilter))
            print(len(mensfilter))    

><
0.2 0.4
0.9090909090909091
11
0.2 0.5
0.9166666666666666
36
0.30000000000000004 0.5
0.9166666666666666
24
0.4 0.5
1.0
14
<<
>>
<>
0.5 0.9
1.0
14


In [306]:
test=mens[(mens['Fav_Serve%']<mens['Dog_Serve%'])&(mens['Dog_Return%']>20)&(mens['Fav_Serve%']>0)&(mens['Dog_Odds']<2.5)]
len(test[test['Winner']==test['Dog']])/len(test)
#len(test)
#test

0.5826771653543307

In [308]:
data=pd.read_sql_query(
    f"Select DISTINCT Winner,Dog,Dog_Record,Winner_Odds,Fav_Record,fav_percent,dog_percent,Fav_Games,Dog_Games,Fav_Odds,Fav,sex,Dog_Odds From results_hard where sex like 'Womens' --where Date > '2023-01-01' and Date not like '{time_now_formatted}'",
    con=devengine,)
data['Fav_Odds']=data['Fav_Odds'].astype(float)
data['Dog_Odds']=data['Dog_Odds'].astype(float)
data['Winx']=data.apply(lambda x: 'Dog' if x['Winner']==x['Dog'] else 'Fav', axis=1)
womensserving=pd.read_csv('womensserving.csv')
combine = pd.merge(data, womensserving, how="left", left_on="Fav", right_on="Name")
combine2 = pd.merge(combine, womensserving, how="left", left_on="Dog", right_on="Name")
combine2[["Service Games Won_x", "Service Games Won_y"]] = combine2[
        ["Service Games Won_x", "Service Games Won_y"]
    ].astype(float)
combine2.rename(
        columns={
            "Service Games Won_x": "Fav_Serve%",
            "Service Games Won_y": "Dog_Serve%",
            "Return Games Won_x": "Fav_Return%",
            "Return Games Won_y": "Dog_Return%",
        },
        inplace=True,
    )
womens = combine2[
    [   "Winner","Dog","Winner_Odds","fav_percent","dog_percent","Fav_Games","Dog_Games","Fav_Odds",
        "Fav",
        # "Player_1_Odds",
        "Fav_Serve%",
        "Dog_Return%",
       
        # "Player_2_Odds",
        "Dog_Serve%",
        "Fav_Return%","Dog_Odds"
    ]]

In [309]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        k=x+0.1
        l=y-0.1
   
        mensfilter=mens[(mens['dog_percent']>=x)&(mens['dog_percent']<=k)&(mens['fav_percent']<=y)&(mens['fav_percent']>=l)&(mens['Dog_Odds']<2.5)]
        mensfiltertxt=f"print(final_hard['Dog'][(final_hard['dog_percent']>{x.round(1)})&(final_hard['dog_percent']<{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})&(final_hard['Dog_Odds']<2.5)])"
        
        if len(mensfilter)>1 and len(mensfilter[mensfilter['Winner']==mensfilter['Dog']])/len(mensfilter) > 0.5:

            print(mensfiltertxt)
   

><
print(final_hard['Dog'][(final_hard['dog_percent']>0.0)&(final_hard['dog_percent']<0.1)&(final_hard['fav_percent']<0.8)&(final_hard['fav_percent']>0.7)&(final_hard['Dog_Odds']<2.5)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.1)&(final_hard['dog_percent']<0.2)&(final_hard['fav_percent']<0.8)&(final_hard['fav_percent']>0.7)&(final_hard['Dog_Odds']<2.5)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.1)&(final_hard['dog_percent']<0.2)&(final_hard['fav_percent']<0.9)&(final_hard['fav_percent']>0.8)&(final_hard['Dog_Odds']<2.5)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.3)&(final_hard['dog_percent']<0.4)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)&(final_hard['Dog_Odds']<2.5)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']<0.7)&(final_hard['fav_percent']<0.3)&(final_hard['fav_percent']>0.2)&(final_hard['Dog_Odds']<2.5)])
print(final_hard['Dog'][(final_hard['dog_percent']>0.6)&(final_hard['dog_percent']

In [310]:
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        #print(x,y)
        k=x+0.2
        l=y-0.2
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['dog_percent']<k)&(womens['fav_percent']<y)&(womens['fav_percent']>l)]
        womensfiltertxt=f"print(final_hard['Dog'][final_hard[(final_hard['Sex']=='womens')&(final_hard['dog_percent']>{x.round(1)})&(final_hard['dog_percent']<{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})])"
        
        if len(womensfilter)>1 and len(womensfilter[womensfilter['Winner']!=womensfilter['Dog']])/len(womensfilter) > 0.9:

            print(womensfiltertxt)
            #print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))

print(final_hard['Dog'][final_hard[(final_hard['Sex']=='womens')&(final_hard['dog_percent']>0.2)&(final_hard['dog_percent']<0.4)&(final_hard['fav_percent']<0.2)&(final_hard['fav_percent']>0.0)])
3
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='womens')&(final_hard['dog_percent']>0.7)&(final_hard['dog_percent']<0.9)&(final_hard['fav_percent']<0.3)&(final_hard['fav_percent']>0.1)])
3
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='womens')&(final_hard['dog_percent']>0.7)&(final_hard['dog_percent']<0.9)&(final_hard['fav_percent']<0.9)&(final_hard['fav_percent']>0.7)])
12
print(final_hard['Dog'][final_hard[(final_hard['Sex']=='womens')&(final_hard['dog_percent']>0.8)&(final_hard['dog_percent']<1.0)&(final_hard['fav_percent']<0.9)&(final_hard['fav_percent']>0.7)])
3


In [311]:
import numpy as np

print("><")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['fav_percent']<y)]
        
        if len(womensfilter)>5 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:

            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))
print("<<")            
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
   
        womensfilter=womens[(womens['dog_percent']<x)&(womens['fav_percent']<y)]
        
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))      
print(">>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['dog_percent']>x)&(womens['fav_percent']>y)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))       
print("<>")
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        
   
        womensfilter=womens[(womens['dog_percent']<x)&(womens['fav_percent']>y)]
        if len(womensfilter)>10 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:
            print(x,y)
            print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            print(len(womensfilter))    

><
0.8 0.6000000000000001
0.6363636363636364
11
0.8 0.7000000000000001
0.6
15
<<
>>
<>


In [312]:
test=womens[(womens['Fav_Return%']>womens['Dog_Return%'])]#&(womens['Dog_Return%']>10)&(womens['Fav_Serve%']>0)&(womens['Dog_Odds']<2.5)&(womens['Dog_Serve%']>70)]
len(test[test['Winner']!=test['Fav']])/len(test)
#len(test)
#test

0.347781217750258

In [313]:
test=womens[(womens['Dog_Odds']<2.5)&(((womens['dog_percent']>0.8)&(womens['fav_percent']<0.6))|((womens['dog_percent']>0.8)&(womens['fav_percent']<0.7)))]
len(test[test['Winner']!=test['Fav']])/len(test)
len(test)

9

In [314]:
for x in np.arange(0,1,0.1):
    for y in np.arange(0,1,0.1):
        k=x+0.1
        l=y-0.1
   
        womensfilter=womens[(womens['dog_percent']>=x)&(womens['dog_percent']<=k)&(womens['fav_percent']<=y)&(womens['fav_percent']>=l)&(womens['Dog_Odds']<2.5)]
        womensfiltertxt=f"print(final_hard[(final_hard['Sex']=='Womens')&(final_hard['dog_percent']>{x.round(1)})&(final_hard['dog_percent']<{k.round(1)})&(final_hard['fav_percent']<{y.round(1)})&(final_hard['fav_percent']>{l.round(1)})&(final_hard['Dog_Odds']<2.5)])"
        
        if len(womensfilter)>1 and len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter) > 0.5:

            print(womensfiltertxt)
            #print(len(womensfilter[womensfilter['Winner']==womensfilter['Dog']])/len(womensfilter))
            #print(len(womensfilter))

print(final_hard[(final_hard['Sex']=='Womens')&(final_hard['dog_percent']>0.0)&(final_hard['dog_percent']<0.1)&(final_hard['fav_percent']<0.5)&(final_hard['fav_percent']>0.4)&(final_hard['Dog_Odds']<2.5)])
print(final_hard[(final_hard['Sex']=='Womens')&(final_hard['dog_percent']>0.0)&(final_hard['dog_percent']<0.1)&(final_hard['fav_percent']<0.6)&(final_hard['fav_percent']>0.5)&(final_hard['Dog_Odds']<2.5)])
print(final_hard[(final_hard['Sex']=='Womens')&(final_hard['dog_percent']>0.1)&(final_hard['dog_percent']<0.2)&(final_hard['fav_percent']<0.2)&(final_hard['fav_percent']>0.1)&(final_hard['Dog_Odds']<2.5)])
print(final_hard[(final_hard['Sex']=='Womens')&(final_hard['dog_percent']>0.1)&(final_hard['dog_percent']<0.2)&(final_hard['fav_percent']<0.4)&(final_hard['fav_percent']>0.3)&(final_hard['Dog_Odds']<2.5)])
print(final_hard[(final_hard['Sex']=='Womens')&(final_hard['dog_percent']>0.1)&(final_hard['dog_percent']<0.2)&(final_hard['fav_percent']<0.5)&(final_hard['fav_percent']>0.4)&(

In [315]:
test=womens[(womens['Fav_Serve%']<womens['Dog_Serve%'])&(womens['Dog_Return%']>20)&(womens['Fav_Return%']<20)&(womens['Fav_Serve%']>0)&(womens['Dog_Odds']<2.5)]
len(test[test['Winner']==test['Dog']])/len(test)
len(test)
#test

7

In [324]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
# Load the data into a Pandas DataFrame

# Load the data into a Pandas DataFrame
data = womens[['Fav_Odds','Dog_Odds','Fav_Serve%','Dog_Serve%','Fav_Return%','Dog_Return%','fav_percent','dog_percent','Winner','Fav','Dog']].copy()
data=data.dropna()
player_1=data.copy()
columns_to_drop=player_1.columns
player_1['Odds']=player_1['Fav_Odds']
player_1['Serve_percent']=player_1['Fav_Serve%']/100
player_1['Return_percent']=player_1['Fav_Return%']/100
player_1['Win_percent']=player_1['fav_percent']
player_1['Win']=player_1.apply(lambda x: 1 if x['Winner']==x['Fav'] else 0, axis=1)
player_1=player_1.drop(columns=columns_to_drop)

player_2=data.copy()
columns_to_drop=player_2.columns
player_2['Odds']=player_2['Dog_Odds']
player_2['Serve_percent']=player_2['Dog_Serve%']/100
player_2['Return_percent']=player_2['Dog_Return%']/100
player_2['Win_percent']=player_2['dog_percent']
player_2['Win']=player_2.apply(lambda x: 1 if x['Winner']!=x['Fav'] else 0, axis=1)
player_2=player_2.drop(columns=columns_to_drop)




train=player_2

# Split the data into training and testing sets
X = train.drop(columns=['Win'])
y = train['Win']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model's performance on the testing data
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.6459627329192547


In [325]:
train

,Odds,Serve_percent,Return_percent,Win_percent,Win
1,2.06,0.631,0.417,0.461538,1
2,4.37,0.538,0.300,0.428571,0
3,2.36,0.638,0.396,0.272727,0
5,2.35,0.671,0.318,0.846154,1
6,2.03,0.718,0.270,0.515152,1
...,...,...,...,...,...
1854,2.22,0.734,0.396,0.300000,0
1855,2.41,0.698,0.314,0.250000,0
1856,2.63,0.771,0.316,0.600000,0
1858,2.81,0.672,0.379,0.562500,0


In [327]:
df = pd.DataFrame(y_pred,columns=['Winner'])
df.to_clipboard()
y_test.to_clipboard()